In [1]:
with open('data/input_19.txt') as fh:
    file_input = fh.read().strip()

In [7]:
test_input = """#ip 0
seti 5 0 1
seti 6 0 2
addi 0 1 0
addr 1 2 3
setr 1 0 0
seti 8 0 4
seti 9 0 5"""

In [8]:
ops = {
    'addr': lambda a, b: (reg[a] + reg[b]),
    'addi': lambda a, b: (reg[a] + b),
    'mulr': lambda a, b: (reg[a] * reg[b]),
    'muli': lambda a, b: (reg[a] * b),
    'banr': lambda a, b: (reg[a] & reg[b]),
    'bani': lambda a, b: (reg[a] & b),
    'borr': lambda a, b: (reg[a] | reg[b]),
    'bori': lambda a, b: (reg[a] | b),
    'setr': lambda a, b: (reg[a]),
    'seti': lambda a, b: (a),
    'gtir': lambda a, b: (1 if a > reg[b] else 0),
    'gtri': lambda a, b: (1 if reg[a] > b else 0),
    'gtrr': lambda a, b: (1 if reg[a] > reg[b] else 0),
    'eqir': lambda a, b: (1 if a == reg[b] else 0),
    'eqri': lambda a, b: (1 if reg[a] == b else 0),
    'eqrr': lambda a, b: (1 if reg[a] == reg[b] else 0),
}

In [9]:
test_input.split('\n')

['#ip 0',
 'seti 5 0 1',
 'seti 6 0 2',
 'addi 0 1 0',
 'addr 1 2 3',
 'setr 1 0 0',
 'seti 8 0 4',
 'seti 9 0 5']

In [119]:
class VM(object):
    def __init__(self, bound=None, debug=False, maxit = -1):
        self.ip = 0
        self.reg = [0, 0, 0, 0, 0, 0]
        self.bound = bound
        self.mem = []
        self.debug = debug
        self.maxit = maxit
        
    def operations(self, ins, *args):
        ops = {
            'addr': lambda a, b: (self.reg[a] + self.reg[b]),
            'addi': lambda a, b: (self.reg[a] + b),
            'mulr': lambda a, b: (self.reg[a] * self.reg[b]),
            'muli': lambda a, b: (self.reg[a] * b),
            'banr': lambda a, b: (self.reg[a] & self.reg[b]),
            'bani': lambda a, b: (self.reg[a] & b),
            'borr': lambda a, b: (self.reg[a] | self.reg[b]),
            'bori': lambda a, b: (self.reg[a] | b),
            'setr': lambda a, b: (self.reg[a]),
            'seti': lambda a, b: (a),
            'gtir': lambda a, b: (1 if a > self.reg[b] else 0),
            'gtri': lambda a, b: (1 if self.reg[a] > b else 0),
            'gtrr': lambda a, b: (1 if self.reg[a] > self.reg[b] else 0),
            'gtrr_mod': lambda a, b: (1 if self.reg[a]**2 > self.reg[b] else 0),
            'eqir': lambda a, b: (1 if a == self.reg[b] else 0),
            'eqri': lambda a, b: (1 if self.reg[a] == b else 0),
            'eqrr': lambda a, b: (1 if self.reg[a] == self.reg[b] else 0),
        }
        if ins == '#ip':
            self.bound = args[0]
        else:
            self.reg[args[2]] = ops[ins](args[0], args[1])
        
        
    def load(self, inp):
        inp = inp.split('\n')
        self.mem = []
        self.ip = 0
        for line in inp:
            args = line.split()
            if len(args) == 4:
                self.mem.append((args[0], int(args[1]), int(args[2]), int(args[3])))
            elif len(args) == 2:
                pass
#                 self.mem.append((args[0], int(args[1]), 0, 0))
    
    def exe(self):
        if self.bound is not None:
            self.reg[self.bound] = self.ip
        ins = self.mem[self.ip]
        before = (self.ip, self.reg[:])
        self.operations(*ins)
        if self.bound is not None:
            self.ip = self.reg[self.bound]
        self.ip += 1
        if self.debug and before[0] == 7:
            print("{:50s} {:20s} {} {}".format(before, ins, (self.ip, self.reg), self.i))
    
    def run(self):
        self.i = 0
        while True:
            self.exe()
            if self.ip < 0 or self.ip >= len(self.mem):
                break
            self.i+=1
            if self.i == self.maxit:
                break

Test

In [65]:
vm = VM(bound=0)
vm.load(test_input)
vm.run()
vm.reg

[6, 5, 6, 0, 0, 9]

Part A

In [ ]:
%%time
vm = VM(bound=1)
vm.load(file_input)
#vm.mem[9] = ('gtrr_mod', 2, 4, 3)
vm.run()
print(vm.reg)

In [120]:
%%time
vm = VM(bound=1, debug=True)
vm.load(file_input)
#vm.mem[9] = ('gtrr_mod', 2, 4, 3)
vm.run()
print(vm.reg)

(7, [0, 7, 861, 1, 861, 1])                        ('addr', 5, 0, 0)    (8, [1, 7, 861, 1, 861, 1]) 6896
(7, [1, 7, 287, 1, 861, 3])                        ('addr', 5, 0, 0)    (8, [4, 7, 287, 1, 861, 3]) 16088
(7, [4, 7, 123, 1, 861, 7])                        ('addr', 5, 0, 0)    (8, [11, 7, 123, 1, 861, 7]) 42344
(7, [11, 7, 41, 1, 861, 21])                       ('addr', 5, 0, 0)    (8, [32, 7, 41, 1, 861, 21]) 138176
(7, [32, 7, 21, 1, 861, 41])                       ('addr', 5, 0, 0)    (8, [73, 7, 21, 1, 861, 41]) 275856
(7, [73, 7, 7, 1, 861, 123])                       ('addr', 5, 0, 0)    (8, [196, 7, 7, 1, 861, 123]) 840888
(7, [196, 7, 3, 1, 861, 287])                      ('addr', 5, 0, 0)    (8, [483, 7, 3, 1, 861, 287]) 1971144
(7, [483, 7, 1, 1, 861, 861])                      ('addr', 5, 0, 0)    (8, [1344, 7, 1, 1, 861, 861]) 5927136
[1344, 256, 862, 1, 861, 862]
CPU times: user 27.1 s, sys: 340 ms, total: 27.5 s
Wall time: 28.3 s


Part B

In [104]:
for i, line in enumerate(file_input.split('\n')[1:]):
    print('{:3d} {}'.format(i, line))

  0 addi 1 16 1
  1 seti 1 1 5
  2 seti 1 4 2
  3 mulr 5 2 3
  4 eqrr 3 4 3
  5 addr 3 1 1
  6 addi 1 1 1
  7 addr 5 0 0
  8 addi 2 1 2
  9 gtrr 2 4 3
 10 addr 1 3 1
 11 seti 2 7 1
 12 addi 5 1 5
 13 gtrr 5 4 3
 14 addr 3 1 1
 15 seti 1 8 1
 16 mulr 1 1 1
 17 addi 4 2 4
 18 mulr 4 4 4
 19 mulr 1 4 4
 20 muli 4 11 4
 21 addi 3 1 3
 22 mulr 3 1 3
 23 addi 3 3 3
 24 addr 4 3 4
 25 addr 1 0 1
 26 seti 0 3 1
 27 setr 1 1 3
 28 mulr 3 1 3
 29 addr 1 3 3
 30 mulr 1 3 3
 31 muli 3 14 3
 32 mulr 3 1 3
 33 addr 4 3 4
 34 seti 0 9 0
 35 seti 0 4 1


In [137]:
%%time
vm = VM(bound=1, debug=True, maxit=20)
vm.reg[0] = 1
vm.load(file_input)
vm.run()
print(vm.reg)

[0, 1, 0, 10550400, 10551261, 1]
CPU times: user 802 µs, sys: 629 µs, total: 1.43 ms
Wall time: 800 µs


In [138]:
def divisors(n):
    res = [n]
    for i in range(1, n):
        if (n % i) == 0:
            res.append(i)
    return res

In [139]:
sum(divisors(861))

1344

Part B solution

In [140]:
sum(divisors(vm.reg[4]))

16078144